In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
data = pd.read_feather('target_stocks.feather')
data.tail(20)

,index,stock_list
139,2024-05-22,[3019]
140,2024-05-23,[00662]
141,2024-05-30,"[2451, 5536, 6285, 2524]"
142,2024-06-03,[3596]
143,2024-06-11,[3532]
144,2024-06-14,[00662]
145,2024-06-21,[6023]
146,2024-06-24,"[2801, 5234]"
147,2024-07-02,"[3189, 6213, 2539]"
148,2024-07-04,"[3023, 2897, 2801]"


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import logging
import yaml
from full_bt import (
    load_config,
    read_stock_data,
    construct_data,
    execute_short_strategy,
    calculate_daily_performance, 
    backtest_single_stock, 
    backtest_wrapper, 
    backtest_multiple_stocks,
    get_stocks_for_date, 
    backtest_multiple_stocks, 
    compute_aggregated_metrics, 
    save_combined_results, 
    main
)


In [4]:
config = load_config('config.yaml')

In [5]:
df= read_stock_data('2539', datetime.strptime('2024-7-17', '%Y-%m-%d'), '/Users/mouyasushi/k_data/永豐')
df.head()

,Open,High,Low,Close,Volume,Amount
ts,,,,,,
2024-07-17 09:01:00,78.8,78.8,78.8,78.8,15,1182000.0
2024-07-17 09:02:00,78.9,79.0,78.7,78.7,17,1340700.0
2024-07-17 09:03:00,78.7,78.7,78.7,78.7,5,393500.0
2024-07-17 09:04:00,78.8,78.8,78.1,78.1,30,2358800.0
2024-07-17 09:06:00,78.6,79.1,78.3,78.8,22,1733800.0


In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
np_data= construct_data(df, ma_period=14) # this is a dictioinary 
close_np = np_data['Close']
print(close_np)
print(len(close_np))


[78.8 78.7 78.7 78.1  0.  78.8 78.7 78.7 78.  77.4 77.4 77.7 77.7 77.9
 77.9  0.  78.3 78.6 77.8 78.6  0.   0.  78.4  0.  78.6  0.  78.5 78.5
  0.  78.5  0.  78.5 78.6 79.  79.  79.2 79.2 79.3 79.2 79.1  0.  79.
 78.8  0.   0.   0.  79.   0.   0.   0.   0.   0.  79.4 79.4 79.3  0.
 79.4 79.3  0.  79.5 79.6  0.  79.6 79.9 79.9  0.  80.1 80.2 80.3  0.
 80.5 80.6 80.4 80.2 80.1  0.  80.1 80.5  0.  80.4 80.3 80.4  0.  80.4
 80.6 80.4 80.3 80.2  0.   0.   0.  80.2 80.2  0.  80.2 80.2 80.1  0.
 80.1  0.   0.  79.9  0.  80.  79.8  0.   0.   0.   0.  80.   0.   0.
  0.   0.  80.   0.   0.  80.   0.   0.   0.   0.   0.   0.  80.1 80.2
 80.2 80.1 80.   0.   0.  80.2  0.  80.3 80.4  0.   0.   0.   0.   0.
  0.   0.  80.4 80.4  0.  80.4  0.  80.3 80.3 80.3  0.   0.   0.  80.2
 80.2  0.  80.2 80.1 80.1  0.   0.   0.   0.   0.   0.  80.1  0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  79.6  0.
 79.6  0.   0.   0.  79.8 79.8  0.   0.   0.   0.   0.   0.   0.   0.
 79.9  0.   0. 

In [7]:
""" 
execute_short_strategy(open_np, close_np, vwap_np, open_time_np, initial_cap, fee_rate, 
                         stop_loss_pct, threshold, entry_time_min, previous_day_last_close) --> equity, trade times
"""

equity = execute_short_strategy(
    np_data['Open'],
    np_data['Close'],
    np_data['VWAP'],
    np_data['open_time'],
    initial_cap=1000000,
    fee_rate=0.001,
    stop_loss_pct=0.04,
    threshold=1.001,
    entry_time_min=546,  # 9:06 AM
    previous_day_last_close=77)

equity

(array([1000000, 1000000, 1000000, 1000000, 1000000,  996455,  997727,
         997727, 1006633, 1014267, 1014267, 1010450, 1010450, 1007905,
        1007905,  999000, 1002816,  999000, 1009178,  999000,  999000,
         999000, 1001544,  999000,  999000,  999000, 1000272, 1000272,
         999000, 1000272,  999000, 1000272,  999000,  993910,  993910,
         991366,  991366,  990094,  991366,  992638,  999000,  993910,
         996455,  999000,  999000,  999000,  993910,  999000,  999000,
         999000,  999000,  999000,  988821,  988821,  990094,  999000,
         988821,  990094,  999000,  987549,  986277,  999000,  986277,
         982460,  982460,  999000,  979916,  978643,  977371,  999000,
         974826,  973554,  976099,  978643,  979916,  999000,  979916,
         974826,  999000,  976099,  977371,  976099,  999000,  976099,
         973554,  976099,  977371,  978643,  999000,  999000,  999000,
         978643,  978643,  999000,  978643,  978643,  979916,  999000,
      

In [8]:
calculate_daily_performance(equity[0], trade_count=equity[1]) 

{'Total Trades': 1,
 'Total Return': -0.013461999999999974,
 'Max Drawdown': 0.0401403180819252,
 'Win Rate': 0.2825278810408922,
 'Daily Volatility': 0.008178933569953886}

In [9]:
data.head()

,index,stock_list
0,2022-10-21,[1717]
1,2022-10-28,"[6873, 1904]"
2,2022-11-03,[2520]
3,2022-11-04,[6412]
4,2022-11-09,"[6188, 1409]"


In [10]:
print(type(data['index']))
print(type(data['stock_list']))
print(type(data['stock_list'][0]))
print(type(data['stock_list'][0][0]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'str'>


In [11]:
import logging

logging.basicConfig(level=logging.DEBUG)


from datetime import datetime

date_to_check = datetime.strptime('2024-07-02', '%Y-%m-%d')
stocks = get_stocks_for_date('target_stocks.feather', date_to_check)
print(f"Stocks for {date_to_check.date()}: {stocks}")


2024-10-23 21:05:05,030 - INFO - [full_bt.py:263] - get_stocks_for_date - Reading stock list from Parquet file: target_stocks.feather
2024-10-23 21:05:05,143 - ERROR - [full_bt.py:327] - get_stocks_for_date - Error reading stock list: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.


Stocks for 2024-07-02: []


In [12]:
backtest_single_stock('3189', date_to_check, config)

2024-10-23 21:05:05,152 - INFO - [full_bt.py:336] - backtest_single_stock - Running backtest for stock: 3189 on date: 2024-07-02


{'Total Trades': 0,
 'Total Return': 0.0,
 'Max Drawdown': 0.0,
 'Win Rate': 0.0,
 'Daily Volatility': 0.0,
 'Stock': '3189',
 'Date': datetime.date(2024, 7, 2),
 'Equity Curve': array([1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
        1000000, 1000000, 1000000, 100000

In [13]:
stocks

[]

In [14]:
backtest_multiple_stocks(stocks, date_to_check, config)

([], {})

### test script


In [15]:
""" 
the result is normal because we take a month to form a monthly return. 
"""

import numpy as np
from datetime import datetime

# Generate mock backtesting results data (daily returns for ~400 days)
# Dates will cover more than one year

mock_data = {
    'Date': pd.date_range(start='2022-01-01', periods=400, freq='D'),  # 400 days starting from 2022-01-01
    'Total Return': np.random.uniform(-0.03, 0.03, 400)  # Random daily returns between -3% and +3%
}

# Convert the dictionary to a DataFrame
mock_results_df = pd.DataFrame(mock_data)

# Test the function
aggregated_metrics = compute_aggregated_metrics(mock_results_df)

aggregated_metrics.tail()



,Date,Weekly Return,Monthly Return,Yearly Return
65,2023-01-31,NaN,-1.651231,NaN
66,2023-02-03,-3.524146,NaN,NaN
67,2023-02-10,0.377358,NaN,NaN
68,2023-02-28,NaN,1.023012,NaN
69,2023-12-31,NaN,NaN,-2.317449


In [16]:
data

,index,stock_list
0,2022-10-21,[1717]
1,2022-10-28,"[6873, 1904]"
2,2022-11-03,[2520]
3,2022-11-04,[6412]
4,2022-11-09,"[6188, 1409]"
...,...,...
154,2024-09-23,[4919]
155,2024-09-24,[2634]
156,2024-09-25,[2606]
157,2024-09-27,[3260]


### New filtered_stock

In [17]:
filtered = pd.read_parquet('filtered_stocks.parquet')

In [18]:
print(filtered.index) # filtered.index
print(filtered['stock_list'])

DatetimeIndex(['2023-10-16', '2023-10-17', '2023-10-18', '2023-10-19',
               '2023-10-20', '2023-10-23', '2023-10-24', '2023-10-25',
               '2023-10-26', '2023-10-27',
               ...
               '2024-09-26', '2024-09-27', '2024-09-30', '2024-10-01',
               '2024-10-04', '2024-10-07', '2024-10-08', '2024-10-09',
               '2024-10-11', '2024-10-14'],
              dtype='datetime64[ns]', name='Date', length=244, freq=None)
Date
2023-10-16                          []
2023-10-17                          []
2023-10-18                          []
2023-10-19                          []
2023-10-20    [3374, 4743, 6901, 2606]
                        ...           
2024-10-07                          []
2024-10-08    [2486, 4961, 2467, 3583]
2024-10-09    [3413, 2363, 2467, 8028]
2024-10-11                      [2363]
2024-10-14                      [2421]
Name: stock_list, Length: 244, dtype: object


In [19]:
stk_list_type = type(filtered['stock_list'][20])
print(stk_list_type)

stk_list = type(filtered['stock_list'][20][2])
print(stk_list)

<class 'numpy.ndarray'>
<class 'str'>


In [20]:
date_to_check = datetime.strptime('2024-07-02', '%Y-%m-%d')

get_stocks_for_date('filtered_stocks.parquet', date_to_check)

2024-10-23 21:05:07,022 - INFO - [full_bt.py:263] - get_stocks_for_date - Reading stock list from Parquet file: filtered_stocks.parquet
2024-10-23 21:05:07,035 - INFO - [full_bt.py:318] - get_stocks_for_date - Found 8 stocks for date 2024-07-02


['3189', '6187', '6442', '2359', '6213', '3583', '5508', '6188']